## Mahcine Translation Encoder Decoder wiht Attetion for SuUAD dataset

In this notebook, Neural Machine Translation with Attention is applied on SQUAD dataset (Standford Question Answering Dataset)

https://rajpurkar.github.io/SQuAD-explorer/

In [1]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-01-08 09:08:31--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   109MB/s    in 0.4s    

2021-01-08 09:08:32 (109 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-01-08 09:08:32--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [4]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [5]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [6]:
import pandas as pd

train_df = pd.DataFrame(list(zip(train_contexts, train_questions, train_answers)),  columns =['contexts', 'questions', 'answers']) 
valid_df = pd.DataFrame(list(zip(val_contexts, val_questions, val_answers)),  columns =['contexts', 'questions', 'answers']) 

In [7]:
train_df['answers'] = train_df['answers'].apply(lambda ans: ans['text'])
valid_df['answers'] = valid_df['answers'].apply(lambda ans: ans['text'])

In [8]:
train_df.head(10)

,contexts,questions,answers
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s
5,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child
6,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,Dangerously in Love
7,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,Mathew Knowles
8,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,late 1990s
9,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,lead singer


In [9]:
import nltk
import string
import re

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer




from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import string
import nltk
import random
import random
#import google_trans_new
#from google_trans_new import google_translator

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
lem = WordNetLemmatizer()
#translator = google_translator()

def clean_text(text):
    ## lower case
    if not isinstance(text, str):
      return str(text) 
    cleaned = text.lower()

    urls_pattern = re.compile(r'https?://\S+|www.\S+')
    cleaned = urls_pattern.sub(r'',cleaned)
    
    ## remove punctuations
    punctuations = string.punctuation
    cleaned_temp = "".join(character for character in cleaned if character not in punctuations)
    
    ## remove stopwords 
    words = cleaned_temp.split()
    #stopword_lists = stopwords.words("english")
    #cleaned = [word for word in words if word not in stopword_lists]
    cleaned = words
    
    ## normalization - lemmatization
    cleaned = [lem.lemmatize(word, "v") for word in cleaned]
    cleaned = [lem.lemmatize(word, "n") for word in cleaned]
    
    ## join 
    cleaned = " ".join(cleaned)
    return cleaned

def back_translate(sentence):
  
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  #print(translations)

  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  # print(translations_en_random)
  return translations_en_random

def random_deletion(sentence, p=0.5): 
    words = sentence.split()
    ret_val = ""
    if len(words) == 1: # return if single word
        ret_val = words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        ret_val = [random.choice(words)] 
    else:
        ret_val = remaining
    return " ".join(ret_val)

def random_swap(sentence, n=5):
    words =  sentence.split()
    length = range(len(words)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        words[idx1], words[idx2] = words[idx2], words[idx1] 
    return " ".join(words)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() 

train_df['contexts'] = train_df['contexts'].progress_apply(lambda txt: clean_text(txt))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [11]:
train_df['contexts']=train_df['contexts'].apply(lambda sent: " ".join([word for word in sent.split()][::-1]) )
train_df['questions']=train_df['questions'].apply(lambda sent: " ".join([word for word in sent.split()][::-1]) )

In [12]:
train_df.head()

,contexts,questions,answers
0,boy baby and love in crazy single numberone 10...,popular? becoming start Beyonce did When,in the late 1990s
1,boy baby and love in crazy single numberone 10...,up? growing was she when in compete Beyonce di...,singing and dancing
2,boy baby and love in crazy single numberone 10...,singer? solo a become and Child Destiny's leav...,2003
3,boy baby and love in crazy single numberone 10...,up? grow Beyonce did state and city what In,"Houston, Texas"
4,boy baby and love in crazy single numberone 10...,famous? become Beyonce did decade which In,late 1990s


In [13]:
%%bash
python -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Preparing Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import torchtext
from torchtext import data 


Then set a random seed for deterministic results/reproducability.

In [15]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Instantiate our German and English spaCy models.

In [16]:
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

In [17]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [18]:
CONTEXT = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

QUESTION = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANSWER = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [19]:
fields = [('context', CONTEXT),('question',QUESTION), ('answer',ANSWER)]

In [20]:
train_field_value_list = list(zip(train_df['contexts'], train_df['questions'],train_df['answers'] ))
valid_field_value_list = list(zip(valid_df['contexts'], valid_df['questions'],valid_df['answers'] ))

In [21]:
train_example = [data.Example.fromlist([SRC_TRG[0], SRC_TRG[1], SRC_TRG[2] ], fields ) for SRC_TRG in train_field_value_list if len(SRC_TRG[0]) > 0 and len(SRC_TRG[1]) > 0 and len(SRC_TRG[2]) > 0 ] 

In [22]:
valid_example = [data.Example.fromlist([SRC_TRG[0], SRC_TRG[1], SRC_TRG[2] ], fields ) for SRC_TRG in valid_field_value_list if len(SRC_TRG[0]) > 0 and len(SRC_TRG[1]) > 0 and len(SRC_TRG[2]) > 0 ] 

Load our data.

In [23]:
train_size = int(len(train_example))
train_idx = list(range(train_size))

In [24]:
val_size = int(0.5* len(valid_example))
valid_example_index_list = list(range(len(valid_example)))
val_idx = valid_example_index_list[0:val_size]
test_idx = valid_example_index_list[val_size:]

In [25]:
train_list = [train_example[i] for i in train_idx ]
val_list = [valid_example[i] for i in val_idx ]
test_list = [valid_example[i] for i in test_idx ]

In [26]:
train = data.Dataset(train_list, fields)
valid = data.Dataset(val_list, fields)
test = data.Dataset(test_list, fields)

We'll also print out an example just to double check they're not reversed.

In [27]:
print(vars(train.examples[0]))

{'context': ['boy', 'baby', 'and', 'love', 'in', 'crazy', 'single', 'numberone', '100', 'hot', 'billboard', 'the', 'feature', 'and', 'award', 'grammy', 'five', 'earn', 'worldwide', 'artist', 'solo', 'a', 'a', 'her', 'establish', 'which', '2003', 'love', 'in', 'dangerously', 'album', 'debut', 'beyoncés', 'of', 'release', 'the', 'saw', 'hiatus', 'their', 'time', 'all', 'of', 'group', 'girl', 'bestselling', 'world', 'the', 'of', 'one', 'become', 'group', 'the', 'knowles', 'mathew', 'father', 'her', 'by', 'manage', 'child', 'destiny', 'girlgroup', 'rb', 'of', 'singer', 'lead', 'a', '1990s', 'late', 'the', 'in', 'fame', 'to', 'rise', 'and', 'child', 'a', 'a', 'competition', 'dance', 'and', 'sing', 'various', 'in', 'perform', 'she', 'texas', 'houston', 'in', 'raise', 'and', 'bear', 'actress', 'and', 'producer', 'record', 'songwriter', 'singer', 'american', 'an', 'be', '1981', '4', 'september', 'bear', 'beeyonsay', 'biːˈjɒnseɪ', 'knowlescarter', 'giselle', 'beyoncé'], 'question': ['popular', 

Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [28]:
#SRC.build_vocab(train, min_freq = 2)
#TRG.build_vocab(train_data, min_freq = 2)

In [29]:
MAX_VOCAB_SIZE = 25_000

CONTEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)

QUESTION.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)

ANSWER.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)



.vector_cache/glove.840B.300d.zip: 2.18GB [16:54, 2.14MB/s]                           
100%|█████████▉| 2195554/2196017 [03:50<00:00, 10403.34it/s]

Finally, define the `device` and create our iterators.

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test), batch_size = BATCH_SIZE, 
                                                           sort_key = lambda x: len(x.context),
                                                           sort_within_batch=True, device = device)                                               


## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [287]:
class Encoder(nn.Module):
    def __init__(self, context_input_dim, context_emb_dim, context_enc_hid_dim, question_input_dim, question_emb_dim, question_enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        #self.hid_dim = hid_dim
        
        self.context_embedding = nn.Embedding(context_input_dim, context_emb_dim)
        
        self.context_rnn = nn.GRU(context_emb_dim, context_enc_hid_dim)
        
        self.context_fc = nn.Linear(context_enc_hid_dim, dec_hid_dim)

        self.question_embedding = nn.Embedding(question_input_dim, question_emb_dim)
        
        self.question_rnn = nn.GRU(question_emb_dim,question_enc_hid_dim)
        
        self.question_fc = nn.Linear(question_enc_hid_dim, dec_hid_dim)

        self.fc = nn.Linear(dec_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.dec_hid_dim = dec_hid_dim
        
    def forward(self, context, question):
        
        #src = [src len, batch size]
        
        context_embedded = self.dropout(self.context_embedding(context))
        
        #embedded = [src len, batch size, emb dim]
        
        context_outputs, context_hidden = self.context_rnn(context_embedded) #no cell state!

        context_hidden = self.context_fc(context_hidden[-1,:,:])


        #src = [src len, batch size]
        
        question_embedded = self.dropout(self.question_embedding(context))
        
        #embedded = [src len, batch size, emb dim]
        
        question_outputs, question_hidden = self.context_rnn(question_embedded) #no cell state!
        question_hidden = self.question_fc(question_hidden[-1,:,:])
        hidden = torch.tanh(self.fc(torch.cat((context_hidden, question_hidden), dim = 1)))
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden



  

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [288]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]


        #context = context.unsqueeze(0)
        #print("After unsqeeuee: context dimenions:", context.size())


        #hidden = hidden.unsqueeze(0)
        #print("After unsqeeuee: hidden dimenions:", hidden.size())
                
        emb_con = torch.cat((embedded, context), dim = 2)

            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)

        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [289]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.dec_hid_dim  == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self,  context, question, answer, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = answer.shape[1]
        trg_len = answer.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context_decoder = self.encoder(context, question)
        context_decoder = context_decoder.unsqueeze(0)
        
        #context also used as the initial hidden state of the decoder
        hidden = context_decoder
        
        #first input to the decoder is the <sos> tokens
        input = answer[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context_decoder)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = answer[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [290]:
CONTEXT_INPUT_DIM = len(CONTEXT.vocab)
QUESTION_INPUT_DIM = len(QUESTION.vocab)
OUTPUT_DIM = len(ANSWER.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(CONTEXT_INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, QUESTION_INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [291]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (context_embedding): Embedding(25004, 300)
    (context_rnn): GRU(300, 256)
    (context_fc): Linear(in_features=256, out_features=256, bias=True)
    (question_embedding): Embedding(25004, 300)
    (question_rnn): GRU(300, 256)
    (question_fc): Linear(in_features=256, out_features=256, bias=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(25004, 300)
    (rnn): GRU(556, 256)
    (fc_out): Linear(in_features=812, out_features=25004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [292]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 44,577,004 trainable parameters


We initiaize our optimizer.

In [293]:
optimizer = optim.Adam(model.parameters(), lr=2e-4 )

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [294]:
context_pretrained_embeddings = CONTEXT.vocab.vectors
print(context_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [295]:
enc.context_embedding.weight.data.copy_(context_pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.1020,  1.6282,  2.1635,  ..., -0.6009, -0.1467,  0.0285],
        ...,
        [-0.1059,  0.0421, -0.0415,  ...,  0.7597,  0.8584,  0.1016],
        [ 0.4681,  0.1534, -0.0751,  ...,  0.6741,  0.0302, -0.7942],
        [ 0.2445, -0.0398,  1.0466,  ..., -0.5515,  0.2743,  0.0540]],
       device='cuda:0')

In [296]:
PAD_IDX = CONTEXT.vocab.stoi[CONTEXT.pad_token]
UNK_IDX = CONTEXT.vocab.stoi[CONTEXT.unk_token]

enc.context_embedding.weight.data[UNK_IDX] = torch.zeros(ENC_EMB_DIM)
enc.context_embedding.weight.data[PAD_IDX] = torch.zeros(ENC_EMB_DIM)

In [297]:
question_pretrained_embeddings = QUESTION.vocab.vectors
print(question_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [298]:
enc.question_embedding.weight.data.copy_(question_pretrained_embeddings)

tensor([[ 1.1717, -0.5923,  0.8272,  ..., -0.2469, -1.2527, -0.8069],
        [-0.8039,  0.1055, -1.1249,  ...,  1.1623,  1.1744, -1.4357],
        [-0.0181, -0.7427,  1.2195,  ...,  0.3657, -0.7106,  2.0183],
        ...,
        [-0.1559,  0.0347,  0.6560,  ..., -0.7573,  0.1280, -0.7493],
        [-0.4331, -0.3423, -0.3420,  ..., -0.1500,  0.2779,  0.2796],
        [-0.0310, -0.4755, -0.3690,  ..., -0.2677, -0.1716,  0.1081]],
       device='cuda:0')

In [299]:
PAD_IDX = QUESTION.vocab.stoi[QUESTION.pad_token]
UNK_IDX = QUESTION.vocab.stoi[QUESTION.unk_token]

enc.question_embedding.weight.data[UNK_IDX] = torch.zeros(ENC_EMB_DIM)
enc.question_embedding.weight.data[PAD_IDX] = torch.zeros(ENC_EMB_DIM)


In [300]:
answer_pretrained_embeddings = ANSWER.vocab.vectors
print(answer_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [301]:
dec.embedding.weight.data.copy_(answer_pretrained_embeddings)

tensor([[-0.2809, -0.0022,  1.4005,  ..., -0.5641,  0.7176,  1.1278],
        [ 0.6511, -0.5002,  0.1395,  ..., -0.4734,  0.2779,  1.0805],
        [ 0.7952,  0.0387,  0.0385,  ...,  0.6222, -1.3961, -0.0756],
        ...,
        [-0.3112,  0.1010, -0.0578,  ..., -0.0196,  0.3005,  0.4144],
        [ 0.7309, -0.0016,  0.9023,  ..., -0.1379, -0.0531, -0.3586],
        [-0.3905, -0.7791,  0.6285,  ...,  0.7102,  0.4332,  0.1823]],
       device='cuda:0')

In [302]:
PAD_IDX = ANSWER.vocab.stoi[ANSWER.pad_token]
UNK_IDX = ANSWER.vocab.stoi[ANSWER.unk_token]

dec.embedding.weight.data[UNK_IDX] = torch.zeros(DEC_EMB_DIM)
dec.embedding.weight.data[PAD_IDX] = torch.zeros(DEC_EMB_DIM)

In [303]:
optimizer = optim.Adam(model.parameters(), lr=2e-4)

In [304]:
TRG_PAD_IDX = ANSWER.vocab.stoi[ANSWER.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [305]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        context = batch.context
        question = batch.question
        answer = batch.answer
        
        optimizer.zero_grad()
        
        output = model(context, question, answer)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        answer = answer[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, answer)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [306]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):


            context = batch.context
            question = batch.question
            answer = batch.answer
            output = model(context, question, answer, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            answer = answer[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, answer)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [307]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [308]:
MODEL_PATH = '/content/drive/My Drive/nlp_end/machine_translation_squad.pth'

In [309]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 53s
	Train Loss: 6.575 | Train PPL: 717.022
	 Val. Loss: 5.920 |  Val. PPL: 372.242
Epoch: 02 | Time: 3m 53s
	Train Loss: 6.239 | Train PPL: 512.555
	 Val. Loss: 5.834 |  Val. PPL: 341.615
Epoch: 03 | Time: 3m 52s
	Train Loss: 5.985 | Train PPL: 397.345
	 Val. Loss: 5.862 |  Val. PPL: 351.342
Epoch: 04 | Time: 3m 51s
	Train Loss: 5.757 | Train PPL: 316.287
	 Val. Loss: 5.882 |  Val. PPL: 358.387
Epoch: 05 | Time: 3m 50s
	Train Loss: 5.571 | Train PPL: 262.780
	 Val. Loss: 5.837 |  Val. PPL: 342.587
Epoch: 06 | Time: 3m 51s
	Train Loss: 5.398 | Train PPL: 220.873
	 Val. Loss: 5.843 |  Val. PPL: 344.909
Epoch: 07 | Time: 3m 51s
	Train Loss: 5.255 | Train PPL: 191.523
	 Val. Loss: 5.862 |  Val. PPL: 351.592
Epoch: 08 | Time: 3m 50s
	Train Loss: 5.110 | Train PPL: 165.698
	 Val. Loss: 5.908 |  Val. PPL: 368.031
Epoch: 09 | Time: 3m 51s
	Train Loss: 4.979 | Train PPL: 145.339
	 Val. Loss: 5.891 |  Val. PPL: 361.628
Epoch: 10 | Time: 3m 51s
	Train Loss: 4.866 | Train PPL

Finally, we test the model on the test set using these "best" parameters.

In [310]:
model.load_state_dict(torch.load(MODEL_PATH))

test_loss = evaluate(model, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.834 | Test PPL: 341.615 |


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.